## Calculate Production Activity at CoolDown T = 0
---
A = Prodx_Rate * (1-exp(-0.693/halflife * beam_time)) * Stop_Ratio
+ Everything in the equation above is in seconds

In [1]:
import pdfplumber
import numpy as np
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment

In [2]:
def Cal_Prodx_A(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames

    df = sheets["Initial"]
    beam_col_index = df.columns.get_loc('Beam')
    run_time_row_idx = df[df.iloc[:, beam_col_index + 2] == 'sec'].index[0]
    run_time = df.iloc[run_time_row_idx, beam_col_index + 1]

    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Prodx A(Bq)" not in df.columns:
                df["Prodx_A(Bq)"] = -1.0
            df.at[df.index[-1], "Prodx A(Bq)"] = np.nan
            for idx in range(len(df)-1):
                prodx_rate = df.at[idx,'Prodx_Rate(pps)']
                halflife = df.at[idx,'t1/2(s)']
                stop_ratio = df.at[idx,'Stop_Ratio']
                df.at[idx,"Prodx_A(Bq)"] = prodx_rate * (1 - np.exp(-0.693/halflife * run_time)) * stop_ratio

            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Summarize sub-sheets

In [71]:
def summarize(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)

    # Extract data from sheets named "*um_*MeV"
    data_frames = []
    for sheet_name in workbook.sheetnames:
        if 'um_' in sheet_name and 'MeV' in sheet_name:
            df = sheets[sheet_name][['A', 'isotope', 'Prodx_A(Bq)']]
            df.columns = ['A', 'isotope', f'A(Bq)@{sheet_name.split("um_")[0]}um']
            data_frames.append((int(sheet_name.split('um_')[0]), df))

    # Sort data_frames by the numeric value in the sheet names
    data_frames.sort(key=lambda x: x[0])

    # Extract only the DataFrames in the correct order
    data_frames = [df for _, df in data_frames]

    # Merge all data frames on 'A' and 'isotope'
    merged_df = data_frames[0]
    for df in data_frames[1:]:
        merged_df = pd.merge(merged_df, df, on=['A', 'isotope'], how='outer')

    # Reorder by "A" from large to small, and then by "isotope" keyword
    merged_df['isotope_keyword'] = merged_df['isotope'].str.replace(' isomer', '')
    merged_df = merged_df.sort_values(by=['A', 'isotope_keyword', 'isotope'], ascending=[False, True, True])
    # Drop the temporary column used for sorting
    merged_df = merged_df.drop(columns=['isotope_keyword'])

    # Round all "Prodx" columns to 2 decimal places
    for col in merged_df.columns:
        if "A(Bq)" in col:
            merged_df[col] = merged_df[col].fillna(0).round(2)

    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        merged_df.to_excel(writer, sheet_name='Summary', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

In [72]:
excel_path = "example/DoesCal.xlsx"
summarize(excel_path)